In [2]:
import os
import ee
import json
import requests
import requests_cache
from pprint import pprint
import pandas as pd

ee.Initialize()

In [6]:
im = ee.Image('projects/mapbiomas-workspace/public/collection2_3/mapbiomas_collection23_integration_v1')

In [2]:
with open('geoms.txt') as json_data:
    d = json.load(json_data)

In [3]:
d[0]

{'adm1': 1,
 'adm2': 1,
 'area_m2': 1595772660.2059546,
 'geom': {'coordinates': [[[[-67.105857849121, -9.68811035156244],
     [-67.0598373413086, -9.70665073394775],
     [-66.8064727783203, -9.81451988220209],
     [-66.6200332641602, -9.89403915405273],
     [-66.588752746582, -9.90319633483881],
     [-66.6233291625975, -9.92320919036865],
     [-66.624137878418, -9.92368125915527],
     [-66.6226501464843, -9.93111991882324],
     [-66.6231842041016, -9.93480110168457],
     [-66.6239166259766, -9.93502902984613],
     [-66.629638671875, -9.93681144714355],
     [-66.6305618286132, -9.93710994720459],
     [-66.6338577270508, -9.93819904327381],
     [-66.6382827758788, -9.95214080810547],
     [-66.6506805419921, -9.94551086425776],
     [-66.6510696411132, -9.94303989410395],
     [-66.6551971435547, -9.94840145111084],
     [-66.6661300659179, -9.95224952697754],
     [-66.6703033447266, -9.96024131774897],
     [-66.6768264770507, -9.95738983154297],
     [-66.676643371582, -

In [4]:
def getAreaByPixels(image, geom):
    dict = image.reduceRegion(reducer=ee.Reducer.frequencyHistogram(),
                              geometry=geom, scale=30, maxPixels=1e12,bestEffort=True)
    data = image.set(dict).getInfo()
    total_pixels = 0
    
    for key, value in data.get('properties').get('classification_2016').items():
        total_pixels += value
    
    tmp = []
    for year in range(2000,2017):
        breakdown = []
        for key, value in data.get('properties').get(f'classification_{year}').items():
            breakdown.append({

                'classification': key,
                'area_%': 100*value/total_pixels,
                'area': value*30*30
            })
        breakdown = sorted(breakdown, key=lambda k: k.get('area'), reverse=True) 
        tmp.append({'classification_year': year, 'data': breakdown})
    
    return tmp

In [28]:
%%time
# classification_data = []
for data in d[4894:]:
    tmp = {
        'iso': 'BRA',
        'adm1': data.get('adm1'),
        'adm2': data.get('adm2'),
#         'total_area_m2': data.get('area_m2'),
#         'geom': data.get('geom'),
        'classification': getAreaByPixels(im, data.get('geom'))
    }
    classification_data.append(tmp)
    
print('done!')

done!
CPU times: user 7.72 s, sys: 601 ms, total: 8.32 s
Wall time: 34min 59s


In [29]:
len(classification_data) #5502 when complete!

5501

In [30]:
with open('classification_data.txt', 'w') as file:
    json.dump(classification_data, file)
print('done')

done


In [3]:
with open('classification_data.txt') as json_data:
    tmp_data = json.load(json_data)

In [4]:
carto_export = []
carto_export.append(['iso', 'adm1', 'adm2',
                     'class2000', 'class2001','class2002','class2003','class2004','class2005', 
                     'class2006','class2007','class2008','class2009','class2010','class2011', 
                     'class2012','class2013','class2014','class2015','class2016'])
for d in tmp_data:
    tmp = []
    tmp.append(d.get('iso')) #iso
    tmp.append(d.get('adm1')) #adm1
    tmp.append(d.get('adm2')) #adm2
#     tmp.append(d.get('geom')) #geom
#     tmp.append(d.get('total_area_m2')) #total_area
    for c in d.get('classification'):
        tmp.append(c.get('data'))
    
    carto_export.append(tmp)

print('done')

done


In [5]:
len(carto_export) #len 5502 if complete

5502

In [8]:
my_df = pd.DataFrame(carto_export)
my_df.to_csv('carto_export.csv', index=False, header=False)